In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### <center> Задание:

1. Данные представлены двумя файлами, для начала их нужно объединить в один датасет 
2. Необходимо провести анализ и очистку "грязных данных", найти аномалии и выбросы, подумать, как их лучше обработать (скорректировать значения, удалить, оставить и обработать отдельно). Для анализа данных и поиска аномалий рекомендуется использовать в том числе и визуализацию
3. В данных есть пропуски, подумать, как их лучше обработать. Если заполнять, то чем именно лучше заполнить в каждом отдельном случае
4. В конце ноутбука заполнить ячейку **"Отчет по анализу"**, в которой по каждому столбцу описать, ***какие именно аномалии были в нем найдены*** (например, выбросы, пропуски, логичесткие противоречия и т.д.) и как ***обработаны*** (удалены, заменены другим значением и т.д.).  
    
**Важно!**
Нужно постараться **найти и исправить** максимальное количество "косяков" в данных, чтобы на выходе получить чистый набор данных, с которым можно дальше работать. Важно не просто найти, но и обработать "косяк", если он не обработан, то зачтен не будет.

**Максимальное количество баллов за работу - 26**

**Описание данных:**

***data.csv*** 
- **id:** id транзакции
- **timestamp:** дата продажи (транзакции)
- **full_sq:** общая площадь 
- **life_sq:** жилая площадь 
- **floor:** этаж 
- **max_floor:** количество этажей в здании
- **material:** материал, из которого изговолены стены 
- **build_year:** год строительства
- **num_room:** количество жилых комнат
- **kitch_sq:** площадь кухни
- **full_all:** количество населения в регионе
- **state:** жилищные условия
- **sub_area:** название территории
- **price_doc:** цена квартиры (целевая переменная)

***macro.csv***
- **timestamp** : дата, на которую актуальны макроэкономические показатели
- **salary** : средняя зарплата в регионе
- **fixed_basket**: стоимость потребительской корзины
- **rent_price_3room_eco**: стоимость аренды 3-х комнатного жилья эконом-класса
- **rent_price_2room_eco**: стоимость аренды 2-х комнатного жилья эконом-класса
- **rent_price_1room_eco**: стоимость аренды 1-но комнатного жилья эконом-класса
- **average_life_exp**: средняя продолжительность жизни в регионе

In [2]:
data = pd.read_csv('data.csv')
macro = pd.read_csv('data_macro.csv')

In [3]:
df = data.merge(macro)
df

,id,timestamp,full_sq,life_sq,floor,state,max_floor,material,build_year,num_room,kitch_sq,full_all,sub_area,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp
0,8059,2013-05-21,11,11.0,2.0,3.0,5.0,2.0,1907.0,1.0,12.0,75377,Hamovniki,2750000,55485.2,14983.92,47.35,40.77,2.31,76.37
1,8050,2013-05-21,63,63.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,247469,Nekrasovka,6426541,55485.2,14983.92,47.35,40.77,2.31,76.37
2,8049,2013-05-21,101,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,7341,Poselenie Moskovskij,8803303,55485.2,14983.92,47.35,40.77,2.31,76.37
3,8055,2013-05-21,37,36.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,13890,Poselenie Sosenskoe,3503310,55485.2,14983.92,47.35,40.77,2.31,76.37
4,8057,2013-05-21,92,91.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,21155,Mitino,13126042,55485.2,14983.92,47.35,40.77,2.31,76.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18861,1992,2012-04-24,57,39.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,118945,Vostochnoe Izmajlovo,7800000,48830.4,13292.33,44.25,37.85,30.89,75.74
18862,500,2011-11-29,34,19.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,26943,Mar'ina Roshha,6800000,44898.7,12959.49,46.50,38.62,31.53,75.79
18863,669,2011-12-20,36,21.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,57405,Preobrazhenskoe,5750000,44898.7,12992.44,46.44,37.93,30.74,75.79
18864,2829,2012-07-12,58,58.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,13890,Poselenie Sosenskoe,5829000,48830.4,13762.47,43.42,38.25,31.10,75.74


In [4]:
df.describe()

,id,full_sq,life_sq,floor,state,max_floor,material,num_room,kitch_sq,full_all,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp
count,18866.000000,18866.000000,17291.000000,18838.000000,15505.000000,17445.000000,17445.000000,17445.000000,17445.000000,1.886600e+04,1.886600e+04,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000
mean,18799.788614,53.940899,34.259152,7.293715,2.201290,12.423330,1.890112,1.934824,7.206420,1.597157e+05,7.582436e+06,59882.202147,16214.145809,49.840684,41.676655,33.376801,76.599302
std,7082.428695,44.116185,59.944591,5.191169,0.852846,6.279832,1.506052,0.864166,26.981615,3.043511e+05,4.983763e+06,3718.214092,1163.579299,1.843967,2.405770,2.346476,0.229903
min,9.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.546000e+03,1.000000e+05,44898.700000,12856.970000,43.420000,0.100000,2.310000,75.740000
25%,13513.250000,38.000000,20.000000,3.000000,2.000000,9.000000,1.000000,1.000000,5.000000,4.843900e+04,5.100000e+06,55485.200000,15417.340000,48.700000,41.170000,32.960000,76.370000
50%,19043.500000,49.000000,30.000000,6.000000,2.000000,12.000000,1.000000,2.000000,7.000000,8.771300e+04,6.685548e+06,61208.000000,16165.300000,49.750000,41.910000,33.650000,76.700000
75%,24748.750000,63.000000,43.000000,10.000000,3.000000,17.000000,2.000000,3.000000,9.000000,1.253540e+05,8.900000e+06,61208.000000,16648.700000,50.930000,42.640000,34.350000,76.700000
max,30473.000000,5326.000000,7478.000000,77.000000,33.000000,57.000000,6.000000,19.000000,2014.000000,1.716730e+06,9.512250e+07,64310.000000,18745.280000,53.210000,43.850000,35.370000,76.770000


In [5]:
df.isna().any()

id                      False
timestamp               False
full_sq                 False
life_sq                  True
floor                    True
state                    True
max_floor                True
material                 True
build_year               True
num_room                 True
kitch_sq                 True
full_all                False
sub_area                False
price_doc               False
salary                  False
fixed_basket            False
rent_price_3room_eco    False
rent_price_2room_eco    False
rent_price_1room_eco    False
average_life_exp        False
dtype: bool

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18866 entries, 0 to 18865
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    18866 non-null  int64  
 1   timestamp             18866 non-null  object 
 2   full_sq               18866 non-null  int64  
 3   life_sq               17291 non-null  float64
 4   floor                 18838 non-null  float64
 5   state                 15505 non-null  float64
 6   max_floor             17445 non-null  float64
 7   material              17445 non-null  float64
 8   build_year            16866 non-null  object 
 9   num_room              17445 non-null  float64
 10  kitch_sq              17445 non-null  float64
 11  full_all              18866 non-null  int64  
 12  sub_area              18866 non-null  object 
 13  price_doc             18866 non-null  int64  
 14  salary                18866 non-null  float64
 15  fixed_basket       

# **Колонка 'timestamp'**

In [7]:
df['timestamp'].unique()

array(['2013-05-21', '2013-05-25', '2013-05-27', ..., '2011-12-20',
       '2012-07-12', '2012-10-16'], dtype=object)

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timestamp']

0       2013-05-21
1       2013-05-21
2       2013-05-21
3       2013-05-21
4       2013-05-21
           ...    
18861   2012-04-24
18862   2011-11-29
18863   2011-12-20
18864   2012-07-12
18865   2012-10-16
Name: timestamp, Length: 18866, dtype: datetime64[ns]

# **Колонка 'state'**



In [9]:
# Проверим странное значение 33
df['state'].unique()

array([ 3., nan,  1.,  2.,  4., 33.])

In [10]:
df['state'].mode()

0    2.0
dtype: float64

In [11]:
# Заменяем все пропуски на моду = 2
# Заменяем странное значение 33 (вероятнее всего случайное двойное нажатие, поэтому заменяем на 3)
df['state'] = df['state'].fillna(2)
df[df['state']==33]
df.loc[714, 'state'] = 3
df['state'].unique()

array([3., 2., 1., 4.])

In [12]:
df.describe()

,id,full_sq,life_sq,floor,state,max_floor,material,num_room,kitch_sq,full_all,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp
count,18866.000000,18866.000000,17291.000000,18838.000000,18866.000000,17445.000000,17445.000000,17445.000000,17445.000000,1.886600e+04,1.886600e+04,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000
mean,18799.788614,53.940899,34.259152,7.293715,2.163840,12.423330,1.890112,1.934824,7.206420,1.597157e+05,7.582436e+06,59882.202147,16214.145809,49.840684,41.676655,33.376801,76.599302
std,7082.428695,44.116185,59.944591,5.191169,0.743862,6.279832,1.506052,0.864166,26.981615,3.043511e+05,4.983763e+06,3718.214092,1163.579299,1.843967,2.405770,2.346476,0.229903
min,9.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.546000e+03,1.000000e+05,44898.700000,12856.970000,43.420000,0.100000,2.310000,75.740000
25%,13513.250000,38.000000,20.000000,3.000000,2.000000,9.000000,1.000000,1.000000,5.000000,4.843900e+04,5.100000e+06,55485.200000,15417.340000,48.700000,41.170000,32.960000,76.370000
50%,19043.500000,49.000000,30.000000,6.000000,2.000000,12.000000,1.000000,2.000000,7.000000,8.771300e+04,6.685548e+06,61208.000000,16165.300000,49.750000,41.910000,33.650000,76.700000
75%,24748.750000,63.000000,43.000000,10.000000,3.000000,17.000000,2.000000,3.000000,9.000000,1.253540e+05,8.900000e+06,61208.000000,16648.700000,50.930000,42.640000,34.350000,76.700000
max,30473.000000,5326.000000,7478.000000,77.000000,4.000000,57.000000,6.000000,19.000000,2014.000000,1.716730e+06,9.512250e+07,64310.000000,18745.280000,53.210000,43.850000,35.370000,76.770000


# **Колонка 'full_sq'**

In [13]:
df['full_sq'].unique()

array([  11,   63,  101,   37,   92,   53,   42,   35,   77,   45,   61,
         52,   82,   38,   40,   48,   74,   62,   57,   60,   93,   51,
         83,   43,   54,   44,   75,   32,   56,   31,   78,   29,   46,
         79,   39,   41,   59,   33,   55,   58,   65,   71,   30,  114,
         87,   36,  102,   66,   64,   49,  165,   69,   50,  110,  142,
         34,   81,  106,  123,  100,   91,   70,   76,   90,   85,   95,
         47,   23,   67,   73,  136,   84,   99,   28,   14,   68,   80,
         72,   20,   17,   24,  103,  119,   96,   97,   86,  125,  138,
         27,  118,  121,  139,  133,   89,   21,   94,  104,  132,  126,
        155,  116,  143,  117,  112,   19,   25,  113,  107,  122,  144,
          1,  202,  134,   98,  120,  167,   22,  124,  115,   26,  137,
        108,  109,   88,  160,  105,   16,  111,  131,  183,   18,  135,
        215,  174,  149,  146,  127,  153,  303,  189,  172,  186,  220,
        181,  141,  161,  219,  148,    0,  154,  1

In [14]:
for i in range(0,18866,1):
  if df['full_sq'].loc[i]<30:
     df['full_sq'].loc[i]=30

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


# **Колонка 'life_sq'**

Не успел

# **Колонка 'kitch_sq'**

In [15]:
df['kitch_sq'].fillna(0, inplace=True)
df.describe()

,id,full_sq,life_sq,floor,state,max_floor,material,num_room,kitch_sq,full_all,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp
count,18866.000000,18866.000000,17291.000000,18838.000000,18866.000000,17445.000000,17445.000000,17445.000000,18866.000000,1.886600e+04,1.886600e+04,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000
mean,18799.788614,54.083324,34.259152,7.293715,2.163840,12.423330,1.890112,1.934824,6.663628,1.597157e+05,7.582436e+06,59882.202147,16214.145809,49.840684,41.676655,33.376801,76.599302
std,7082.428695,44.014858,59.944591,5.191169,0.743862,6.279832,1.506052,0.864166,26.015145,3.043511e+05,4.983763e+06,3718.214092,1163.579299,1.843967,2.405770,2.346476,0.229903
min,9.000000,30.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.546000e+03,1.000000e+05,44898.700000,12856.970000,43.420000,0.100000,2.310000,75.740000
25%,13513.250000,38.000000,20.000000,3.000000,2.000000,9.000000,1.000000,1.000000,5.000000,4.843900e+04,5.100000e+06,55485.200000,15417.340000,48.700000,41.170000,32.960000,76.370000
50%,19043.500000,49.000000,30.000000,6.000000,2.000000,12.000000,1.000000,2.000000,6.000000,8.771300e+04,6.685548e+06,61208.000000,16165.300000,49.750000,41.910000,33.650000,76.700000
75%,24748.750000,63.000000,43.000000,10.000000,3.000000,17.000000,2.000000,3.000000,9.000000,1.253540e+05,8.900000e+06,61208.000000,16648.700000,50.930000,42.640000,34.350000,76.700000
max,30473.000000,5326.000000,7478.000000,77.000000,4.000000,57.000000,6.000000,19.000000,2014.000000,1.716730e+06,9.512250e+07,64310.000000,18745.280000,53.210000,43.850000,35.370000,76.770000


In [16]:
for i in range(0,18866,1):
  if df['kitch_sq'].loc[i]==0:
     df['full_sq'].loc[i]=5

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [17]:
df['kitch_sq'].head(15)

0     12.0
1      0.0
2      0.0
3      0.0
4      0.0
5      8.0
6      0.0
7      0.0
8     12.0
9      6.0
10     0.0
11     0.0
12     0.0
13     8.0
14     1.0
Name: kitch_sq, dtype: float64

# **Колонка 'floor'**

In [18]:
df['floor'].unique()

array([ 2.,  3.,  6., 10.,  5., 24., 11.,  4., 15., 17., 12., 19.,  1.,
        7.,  8., 16., 13.,  9., nan, 14., 22., 32., 20., 18., 25., 23.,
       21., 37., 30., 28., 26.,  0., 31., 36., 34., 41., 77., 27., 29.,
       44., 33.])

In [19]:
df['floor'].fillna(0, inplace=True)

In [20]:
df['floor'].unique()

array([ 2.,  3.,  6., 10.,  5., 24., 11.,  4., 15., 17., 12., 19.,  1.,
        7.,  8., 16., 13.,  9.,  0., 14., 22., 32., 20., 18., 25., 23.,
       21., 37., 30., 28., 26., 31., 36., 34., 41., 77., 27., 29., 44.,
       33.])

In [21]:
df['floor'].isna().sum()

0

In [22]:
df[df['floor']==77]
df.loc[12018, 'floor'] = 3
df['floor'].unique()

array([ 2.,  3.,  6., 10.,  5., 24., 11.,  4., 15., 17., 12., 19.,  1.,
        7.,  8., 16., 13.,  9.,  0., 14., 22., 32., 20., 18., 25., 23.,
       21., 37., 30., 28., 26., 31., 36., 34., 41., 27., 29., 44., 33.])

In [23]:
df_vibros=df[df['floor'] > df['max_floor']]
df_vibros

,id,timestamp,full_sq,life_sq,floor,state,max_floor,material,build_year,num_room,kitch_sq,full_all,sub_area,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp
22,8271,2013-05-31,93,93.0,3.0,1.0,1.0,1.0,2013.0,3.0,1.0,5740,Poselenie Pervomajskoe,5427640,55485.2,14983.92,47.35,40.77,2.31,76.37
61,8502,2013-06-14,5,18.0,2.0,2.0,0.0,1.0,1979.0,1.0,0.0,86206,Bibirevo,6000000,55485.2,15130.84,46.56,41.09,32.23,76.37
182,8915,2013-07-03,51,30.0,5.0,1.0,0.0,1.0,1997.0,2.0,8.0,41504,Beskudnikovskoe,3850000,55485.2,15163.82,48.08,41.42,32.55,76.37
275,9164,2013-07-12,83,42.0,8.0,2.0,3.0,2.0,1961.0,3.0,9.0,116742,Tverskoe,24435000,55485.2,15163.82,48.08,41.42,32.55,76.37
312,9260,2013-07-17,45,18.0,8.0,1.0,1.0,1.0,2014.0,1.0,1.0,9553,Poselenie Voskresenskoe,4309670,55485.2,15163.82,48.08,41.42,32.55,76.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17512,30276,2015-06-16,5,0.0,12.0,3.0,0.0,1.0,2014.0,3.0,0.0,112221,Chertanovo Juzhnoe,1085877,64310.0,18720.65,49.01,38.91,30.29,76.77
17538,30302,2015-06-17,5,NaN,8.0,2.0,0.0,1.0,NaN,3.0,0.0,13890,Poselenie Sosenskoe,7350340,64310.0,18720.65,49.01,38.91,30.29,76.77
17551,30320,2015-06-18,47,1.0,8.0,1.0,1.0,1.0,2016.0,1.0,1.0,112804,Filevskij Park,9475779,64310.0,18720.65,49.01,38.91,30.29,76.77
17570,30344,2015-06-20,50,1.0,22.0,1.0,1.0,1.0,2015.0,1.0,1.0,67710,Krylatskoe,7350000,64310.0,18720.65,49.01,38.91,30.29,76.77


In [24]:
for i in range(0,18866,1):
  if df['floor'].loc[i] > df['max_floor'].loc[i]:
     df['floor'].loc[i] = df['max_floor'].loc[i]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


**Проблема, что этаж больше максимального этажа. Но этого не может быть. Но я не знаю, как это корректно обработать...Приравнял максимальный этаж к обычному этажу, но это не совсем корректно получается.**

In [25]:
for i in range(0,18866,1):
  if df['floor'].loc[i]==0:
     df['floor'].loc[i] = df['max_floor'].loc[i]
df[df['floor']==0] 

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,id,timestamp,full_sq,life_sq,floor,state,max_floor,material,build_year,num_room,kitch_sq,full_all,sub_area,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp
61,8502,2013-06-14,5,18.0,0.0,2.0,0.0,1.0,1979.0,1.0,0.0,86206,Bibirevo,6000000,55485.2,15130.84,46.56,41.09,32.23,76.37
182,8915,2013-07-03,51,30.0,0.0,1.0,0.0,1.0,1997.0,2.0,8.0,41504,Beskudnikovskoe,3850000,55485.2,15163.82,48.08,41.42,32.55,76.37
364,9426,2013-07-24,61,NaN,0.0,2.0,0.0,1.0,NaN,2.0,7.0,9553,Poselenie Voskresenskoe,5647680,55485.2,15163.82,48.08,41.42,32.55,76.37
725,10089,2013-08-27,5,NaN,0.0,1.0,0.0,4.0,NaN,1.0,0.0,13890,Poselenie Sosenskoe,3591000,55485.2,15241.93,48.22,42.33,33.15,76.37
748,10145,2013-08-29,5,60.0,0.0,1.0,0.0,4.0,0.0,2.0,0.0,247469,Nekrasovka,6518400,55485.2,15241.93,48.22,42.33,33.15,76.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17299,30017,2015-05-28,5,NaN,0.0,1.0,0.0,4.0,0.0,1.0,0.0,102590,Danilovskoe,6500000,64310.0,18745.28,50.42,39.16,31.09,76.77
17378,30114,2015-06-04,53,NaN,0.0,1.0,0.0,1.0,0.0,2.0,1.0,8384,Poselenie Novofedorovskoe,3175380,64310.0,18720.65,49.01,38.91,30.29,76.77
17408,30143,2015-06-05,5,NaN,0.0,2.0,0.0,1.0,NaN,3.0,0.0,122862,Izmajlovo,20000000,64310.0,18720.65,49.01,38.91,30.29,76.77
17512,30276,2015-06-16,5,0.0,0.0,3.0,0.0,1.0,2014.0,3.0,0.0,112221,Chertanovo Juzhnoe,1085877,64310.0,18720.65,49.01,38.91,30.29,76.77


In [26]:
df.describe()

,id,full_sq,life_sq,floor,state,max_floor,material,num_room,kitch_sq,full_all,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp
count,18866.000000,18866.000000,17291.000000,18838.000000,18866.000000,17445.000000,17445.000000,17445.000000,18866.000000,1.886600e+04,1.886600e+04,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000
mean,18799.788614,48.359112,34.259152,6.973139,2.163840,12.423330,1.890112,1.934824,6.663628,1.597157e+05,7.582436e+06,59882.202147,16214.145809,49.840684,41.676655,33.376801,76.599302
std,7082.428695,24.890590,59.944591,5.161389,0.743862,6.279832,1.506052,0.864166,26.015145,3.043511e+05,4.983763e+06,3718.214092,1163.579299,1.843967,2.405770,2.346476,0.229903
min,9.000000,5.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.546000e+03,1.000000e+05,44898.700000,12856.970000,43.420000,0.100000,2.310000,75.740000
25%,13513.250000,37.000000,20.000000,3.000000,2.000000,9.000000,1.000000,1.000000,5.000000,4.843900e+04,5.100000e+06,55485.200000,15417.340000,48.700000,41.170000,32.960000,76.370000
50%,19043.500000,45.000000,30.000000,6.000000,2.000000,12.000000,1.000000,2.000000,6.000000,8.771300e+04,6.685548e+06,61208.000000,16165.300000,49.750000,41.910000,33.650000,76.700000
75%,24748.750000,61.000000,43.000000,10.000000,3.000000,17.000000,2.000000,3.000000,9.000000,1.253540e+05,8.900000e+06,61208.000000,16648.700000,50.930000,42.640000,34.350000,76.700000
max,30473.000000,637.000000,7478.000000,44.000000,4.000000,57.000000,6.000000,19.000000,2014.000000,1.716730e+06,9.512250e+07,64310.000000,18745.280000,53.210000,43.850000,35.370000,76.770000


# **Колонка 'max_floor'**

In [27]:
df['max_floor'].unique()

array([ 5., nan, 16., 17.,  9., 24.,  1., 10., 25., 14., 12.,  0.,  8.,
       18., 23., 20.,  6., 22.,  7.,  4., 30., 11.,  3., 40., 32., 15.,
       27.,  2., 19., 28., 29., 39., 38., 13., 31., 21., 26., 37., 33.,
       47., 36., 34., 48., 42., 57., 44., 45., 41.])

In [28]:
df_vibros=df[df['floor'] > df['max_floor']]
df_vibros

,id,timestamp,full_sq,life_sq,floor,state,max_floor,material,build_year,num_room,kitch_sq,full_all,sub_area,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp


# **Колонка 'material'**


In [29]:
df['material'].mode()

0    1.0
dtype: float64

In [30]:
df['material'].fillna(1, inplace=True)
df.describe()

,id,full_sq,life_sq,floor,state,max_floor,material,num_room,kitch_sq,full_all,price_doc,salary,fixed_basket,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,average_life_exp
count,18866.000000,18866.000000,17291.000000,18838.000000,18866.000000,17445.000000,18866.000000,17445.000000,18866.000000,1.886600e+04,1.886600e+04,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000,18866.000000
mean,18799.788614,48.359112,34.259152,6.973139,2.163840,12.423330,1.823068,1.934824,6.663628,1.597157e+05,7.582436e+06,59882.202147,16214.145809,49.840684,41.676655,33.376801,76.599302
std,7082.428695,24.890590,59.944591,5.161389,0.743862,6.279832,1.467149,0.864166,26.015145,3.043511e+05,4.983763e+06,3718.214092,1163.579299,1.843967,2.405770,2.346476,0.229903
min,9.000000,5.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.546000e+03,1.000000e+05,44898.700000,12856.970000,43.420000,0.100000,2.310000,75.740000
25%,13513.250000,37.000000,20.000000,3.000000,2.000000,9.000000,1.000000,1.000000,5.000000,4.843900e+04,5.100000e+06,55485.200000,15417.340000,48.700000,41.170000,32.960000,76.370000
50%,19043.500000,45.000000,30.000000,6.000000,2.000000,12.000000,1.000000,2.000000,6.000000,8.771300e+04,6.685548e+06,61208.000000,16165.300000,49.750000,41.910000,33.650000,76.700000
75%,24748.750000,61.000000,43.000000,10.000000,3.000000,17.000000,2.000000,3.000000,9.000000,1.253540e+05,8.900000e+06,61208.000000,16648.700000,50.930000,42.640000,34.350000,76.700000
max,30473.000000,637.000000,7478.000000,44.000000,4.000000,57.000000,6.000000,19.000000,2014.000000,1.716730e+06,9.512250e+07,64310.000000,18745.280000,53.210000,43.850000,35.370000,76.770000


**Также, наверное, неправильное исправление. Но так и не смог придумать, как сделать лучше :(**

# **Колонка 'build_year'**

In [31]:
df['build_year'].unique()

array(['1907.0', nan, '1980.0', '2014.0', '1970.0', '1982.0', '2013.0',
       '2004.0', '2003.0', '1957.0', '1986.0', '1960.0', '1995.0',
       '1979.0', '1975.0', '1987.0', '1962.0', '1969.0', '1993.0',
       '1996.0', '1972.0', '2011.0', '1965.0', '2010.0', '1985.0',
       '2006.0', '1961.0', '1971.0', '1978.0', '1966.0', '1967.0',
       '2000.0', '1964.0', '1977.0', '1983.0', '1968.0', '1974.0',
       '2008.0', '1959.0', '2007.0', '1984.0', '1976.0', '1997.0',
       '1989.0', '1958.0', '1988.0', '2012.0', '1990.0', '1946.0',
       '1917.0', '2002.0', '2005.0', '2001.0', '1963.0', '1954.0',
       '1951.0', '1981.0', '1955.0', '1999.0', '2009.0', '1973.0',
       '1994.0', '1998.0', '1992.0', '1950.0', '1956.0', '2015.0', '0.0',
       '1932.0', '1.0', '1937.0', '1938.0', '1939.0', '1991.0', '1934.0',
       '1935.0', '20052009.0', '1947.0', '1953.0', '1933.0', '2016.0',
       '1930.0', '1912.0', '1929.0', '3.0', '1928.0', '1915.0', '1936.0',
       '1925.0', '1940.0', '1943

In [32]:
# Смена буквы о на 0
df['build_year']=df['build_year'].fillna(0)
df=df.replace({'build_year' : { 'o': 0}})

In [33]:
df['build_year'].unique()

array(['1907.0', 0, '1980.0', '2014.0', '1970.0', '1982.0', '2013.0',
       '2004.0', '2003.0', '1957.0', '1986.0', '1960.0', '1995.0',
       '1979.0', '1975.0', '1987.0', '1962.0', '1969.0', '1993.0',
       '1996.0', '1972.0', '2011.0', '1965.0', '2010.0', '1985.0',
       '2006.0', '1961.0', '1971.0', '1978.0', '1966.0', '1967.0',
       '2000.0', '1964.0', '1977.0', '1983.0', '1968.0', '1974.0',
       '2008.0', '1959.0', '2007.0', '1984.0', '1976.0', '1997.0',
       '1989.0', '1958.0', '1988.0', '2012.0', '1990.0', '1946.0',
       '1917.0', '2002.0', '2005.0', '2001.0', '1963.0', '1954.0',
       '1951.0', '1981.0', '1955.0', '1999.0', '2009.0', '1973.0',
       '1994.0', '1998.0', '1992.0', '1950.0', '1956.0', '2015.0', '0.0',
       '1932.0', '1.0', '1937.0', '1938.0', '1939.0', '1991.0', '1934.0',
       '1935.0', '20052009.0', '1947.0', '1953.0', '1933.0', '2016.0',
       '1930.0', '1912.0', '1929.0', '3.0', '1928.0', '1915.0', '1936.0',
       '1925.0', '1940.0', '1943.0

# <center> **Отчет по анализу:**

- **id:** 
- **timestamp:** Тип object сменил на datetime
- **full_sq:** Минимальное значение 0, заменил на 30
- **life_sq:**  Пропуски, Минимальное значение 0 (но не исправил)
- **floor:**  Пропуски, Минимальное значение 0, есть выброс - 77 (2 раза нажали 7)
- **max_floor:** Пропуски, Минимальное значение 0
- **material:**  Пропуски
- **build_year:** Пропуски, буква о вместо 0
- **num_room:** Пропуски, Минимальное значение 0 (также не сделал)
- **kitch_sq:** Пропуски
- **full_all:** 
- **state:** есть пропуски, есть выброс - 33 (2 раза нажали 3 возможно)
- **sub_area:** 
- **price_doc:** 
- **timestamp** :
- **salary** : 
- **fixed_basket**: 
- **rent_price_3room_eco**: 
- **rent_price_2room_eco**: 
- **rent_price_1room_eco**: 
- **average_life_exp**: 